In [1]:
# Importation des librairies
import requests
import pandas as pd
from datetime import datetime, timedelta, date

import plotly.express as px

import scrapy
from scrapy.crawler import CrawlerProcess
import os
import logging
import urllib

import boto3

#### PART 1. Obtenir les coordonnées geographiques des villes

In [2]:
# liste donnée des villes
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
def city_gps(city_name):
    """
    création d'une fonction afin de récupérer au format json les informations sur une ville, à partir de son nom passé en paramètre
    utlisation de l'API du site nominatim avec des critères en paramètre (format de sortie + nom ville + pays)
    utilisation d'un try/except au cas ou certaines noms dans la liste renvois une erreur. Dans ce cas nous passons l'erreur et affectons une valeur vide aux données recherchées
    Args:
        city_name : liste des villes
    Returns:
        Latitude et longitude pour chaque ville sous forme d'un tuple
    """
    url_nominatim = "https://nominatim.openstreetmap.org/search"
    criteria_nominatim = {"format":"json", "city":city_name, "country":"France"}
    r = requests.get(url_nominatim, params=criteria_nominatim)
    r.json()
    try :
        city_lon = r.json()[0]["lon"]
        city_lat = r.json()[0]["lat"]
    except :
        city_lon = ""
        city_lat = ""
    return city_lon, city_lat

In [4]:
# test sur une ville 
city_gps("Marseille")[0]

'5.3699525'

In [5]:
#création d'un dataframe à partir du nom
dataset_cities = pd.DataFrame({"City":cities})

# ajout de deux colonnes avec les coordonnées gps (latitude & longitude)
dataset_cities["Latitude"] = dataset_cities["City"].apply(lambda x: city_gps(x)[1])
dataset_cities["Longitude"] = dataset_cities["City"].apply(lambda x: city_gps(x)[0])

In [6]:
# verification du résultat
dataset_cities

,City,Latitude,Longitude
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658
5,Paris,48.8588897,2.3200410217200766
6,Amiens,49.8941708,2.2956951
7,Lille,50.6365654,3.0635282
8,Strasbourg,48.584614,7.7507127
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


In [7]:
# ajout d'un id pour chaque ville, insertion de la colonne en 1ère position avec la méthode "insert" car sinon l'ajout se fait en fin de dataframe
# cet id sera utile par la suite
dataset_cities.insert(0, "id", [i for i in range(1,36)])

In [8]:
# vérification des résultats
dataset_cities

,id,City,Latitude,Longitude
0,1,Mont Saint Michel,48.6359541,-1.511459954959514
1,2,St Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8588897,2.3200410217200766
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.584614,7.7507127
9,10,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


#### PART 2. Obtenir les données météorologiques concernant les villes

Le site openweathermap renvoi à partir de coordonnées GPS un jeu de données trés etoffés qui comprend des données courantes, et des prévisions par minutes, heures, et jours.
Nous avons fait le choix de ne garder que les données concernant les prévisions journalières, qui sont sur 7 jours.
Les données journalières étant également nombreuses, nous n'avons également retenu que certaines d'entre elles :
- le date du jour en question
- la temperature (en degré celsius)
- le taux d'humidité (e n%)
- la vitesse du vent (en m/seconde)
- la pluie (en mm par jour)

In [9]:
# fonction créée avec l'aide d'exemples trouvés sur internet (notamment stackoverfflow) mais je n'ai malheuresuement pas noté le lien

def read_credentials(file_path):
    """
    fonction qui permet d'aller chercher les codes d'accès à openweahermap dans un fichier txt situé au même emplacement que le script
    Args:
        file_path (string): chemin d'accès et nom du fichier
    Returns:
        un dictionnaire avec les credentials sous forme de clé(s)/valeur(s)
    """
    credentials = {}
    with open(file_path, "r") as file:
        for line in file:
            key, value = line.strip().split("=")
            credentials[key] = value
    return credentials

# Détermination du chemin et/ou nom du fichier où sont les credentials
credentials_path = "access.txt"

# Lecture des credentials à partir du fichier indiqué
credentials = read_credentials(credentials_path)

# Affectation à 1 variable des credentials
app_id = credentials["app_id"]

In [10]:
def city_weather(lat, lon, day, app_id):
    """
    création d'une fonction qui renvoie des infos clés sur la météo prévisionnelle.
    la fonction à trois paramètres : les coordonnées gps (latitude/longitude) et le jour pour lequel nous souhaitons les données
    utlisation de l'API du site openweathermap (owm) avec des critères passés en paramètres : la clé de l'API, les coordonnées GPS, l'exclusion de certaines données (exclude) et le choix des unités (units)
    a noter également deux points :
    i) nous avons transformé la date qui n'est pas au bon format
    ii) et nous avons utilisé un try/except pour gérer l'absence de la donnée sur la pluie liée au fait qu'il ne pleut pas. Dans ce cas nous afffectons la valeur 0

    Args:
        lat (float): latitude d'une ville
        lon (float): longitude d'une ville
        day (int): jour
        app_id (string) : 

    Returns:
        liste comprenant la date du jour, la température, l'humidité, le vent et la pluie
    """
    url_owm = "https://api.openweathermap.org/data/3.0/onecall?"
    criteria_owm = {"lat":lat, "lon":lon, "appid":app_id, "exclude":"current,minutely,hourly", "units":"metric"}
    data_forecast = requests.get(url_owm, params = criteria_owm).json()
    day_date = data_forecast["daily"][day]["dt"]
    day_date = datetime.fromtimestamp(day_date).strftime("%x")
    day_temp = data_forecast["daily"][day]["temp"]["day"]
    day_humidity = data_forecast["daily"][day]["humidity"]
    day_wind = data_forecast["daily"][day]["wind_speed"]
    try :
        day_rain = data_forecast["daily"][day]["rain"]
    except :
        day_rain=0
    return [day_date, day_temp, day_humidity, day_wind, day_rain]

In [11]:
# création d'un dataframe destiné à contenir les prévisions météo à partir des coordonnées gps
# les colonnes correspondent aux données météorologiques prévisionnelles
# les lignes correspondent aux villes et aux jours (donc plusieurs lignes pour une même ville)
dataset_forecast = pd.DataFrame(columns = ["id", "Date", "Temperature", "Humidity", "Wind", "Rain"])
dataset_forecast

,id,Date,Temperature,Humidity,Wind,Rain


In [12]:
# itération sur les villes (35 villes) et les prévisions journalières (7 jours) pour compléter le dataset
# nous aurions pu remplacer les chiffres par une valeur variable en fonction des résultats précédent (len par exemple)
# nous utilisons la fonction city_weather créée précédemment
# nous créons un dataframe temporaire pour y affecter les données renvoyées par la fonction
# nous concaténons ce dataframe temporaire au dataframe principal
for city in range(len(dataset_cities)):
    for i in range(7):
        response = city_weather(dataset_cities["Latitude"][city], dataset_cities["Longitude"][city], i, app_id)
        dataset_forecast_bis = pd.DataFrame({"id":city+1, "Date":response[0], "Temperature":response[1], "Humidity":response[2], "Wind":response[3], "Rain":response[4]}, index=[i+7*city])
        dataset_forecast = pd.concat([dataset_forecast, dataset_forecast_bis], ignore_index=True)

C:\Users\SD\AppData\Local\Temp\ipykernel_28252\1853480214.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_forecast = pd.concat([dataset_forecast, dataset_forecast_bis], ignore_index=True)


In [13]:
# la taille du dataframe permet de confirmer les résultats. nous avons 245 lignes soit 35 villes x 7 jours
dataset_forecast.shape

(245, 6)

In [14]:
# nous avons bien également 7 lignes pour le même id (ville)
dataset_forecast.head(40)

,id,Date,Temperature,Humidity,Wind,Rain
0,1,03/11/24,11.21,82,6.85,0.24
1,1,03/12/24,10.46,96,5.51,2.06
2,1,03/13/24,14.42,72,5.03,0.00
3,1,03/14/24,14.99,72,5.96,0.25
4,1,03/15/24,13.77,83,6.60,0.69
5,1,03/16/24,15.12,79,6.44,3.96
6,1,03/17/24,13.27,82,6.36,1.92
7,2,03/11/24,11.02,80,7.54,0.00
8,2,03/12/24,11.87,88,6.32,1.00
9,2,03/13/24,13.57,73,7.12,0.00


In [15]:
# nous vérifions également le format des données
dataset_forecast.dtypes

id              object
Date            object
Temperature    float64
Humidity        object
Wind           float64
Rain           float64
dtype: object

In [16]:
# certains formats ne sont pas adaptés au type de données, ou dans un format qui prend de la place
# conversion des colonnes numériques avec la fonction pandas to_numeric avec en paramètre downcast ce que l'on souhaite (float)
# conversion de la colonne date avec la fonction pandas to_datetime
dataset_forecast[["Humidity", "Rain", "Temperature", "Wind"]] = dataset_forecast[["Humidity", "Rain", "Temperature", "Wind"]].apply(pd.to_numeric, downcast="float")
dataset_forecast["Date"] = dataset_forecast["Date"].apply(pd.to_datetime)

#verification du résultat
dataset_forecast.dtypes

id                     object
Date           datetime64[ns]
Temperature           float32
Humidity              float32
Wind                  float32
Rain                  float32
dtype: object

Nous avons à ce stade trop de données pour être capable de faire un choix sur une destination.
Nous allons pour chaque critère calculer une valeur unique sur l'ensemble de la période de prévision, pour chaque id (ville).
Nous avons retenu la médiane, plus robuste que la moyenne.

Nous considérerons donc pour la suite du travail que les donnée ssont justes.

In [17]:
# utlisation de la fonction groupby() à partir de l'id. le paramètre as_index permet de garder l'id comme une colonne sinon il passerait en index
# les chiffres sont arrondis
median_forecast = dataset_forecast.groupby(["id"], as_index=False)[["Temperature", "Humidity", "Wind", "Rain"]].median().round(1)
median_forecast

,id,Temperature,Humidity,Wind,Rain
0,1,13.800000,82.0,6.4,0.7
1,2,13.600000,79.0,7.1,0.2
2,3,14.100000,78.0,6.6,0.8
3,4,11.400000,87.0,6.4,1.9
4,5,13.600000,81.0,5.5,1.9
5,6,14.900000,60.0,3.8,0.6
6,7,13.400000,75.0,5.6,0.8
7,8,14.000000,73.0,5.6,1.1
8,9,13.900000,65.0,4.0,1.3
9,10,12.500000,63.0,2.3,0.7


Maintenant que nous avons pour chaque ville (id) une valeur unique par paramètre, nous allons rajouter à notre dataframe initiale les colonnes concernant la météo previsionnelle

In [18]:
# utilisation de la fonction merge() pour fusionner les 2 dataframes en jointant sur la colonne commune "id"
dataset_cities = dataset_cities.merge(median_forecast, on=["id"])
dataset_cities

,id,City,Latitude,Longitude,Temperature,Humidity,Wind,Rain
0,1,Mont Saint Michel,48.6359541,-1.511459954959514,13.800000,82.0,6.4,0.7
1,2,St Malo,48.649518,-2.0260409,13.600000,79.0,7.1,0.2
2,3,Bayeux,49.2764624,-0.7024738,14.100000,78.0,6.6,0.8
3,4,Le Havre,49.4938975,0.1079732,11.400000,87.0,6.4,1.9
4,5,Rouen,49.4404591,1.0939658,13.600000,81.0,5.5,1.9
5,6,Paris,48.8588897,2.3200410217200766,14.900000,60.0,3.8,0.6
6,7,Amiens,49.8941708,2.2956951,13.400000,75.0,5.6,0.8
7,8,Lille,50.6365654,3.0635282,14.000000,73.0,5.6,1.1
8,9,Strasbourg,48.584614,7.7507127,13.900000,65.0,4.0,1.3
9,10,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,12.500000,63.0,2.3,0.7


In [19]:
# sauvegarde du résultat sous forme de fichier csv, sans les index
dataset_cities.to_csv("dataset_cities.csv", index=False)

Selon nos critères (arbitraire pour les besoins de l'exercice), les destinations les plus intéressantes seront celles avec :
- une quantité de pluie inférieure ou égale à 0.5
- une température moyenne supérieure à 10 degrés
- une humidité inférieure ou égale à 60%
- un vent au maximum de 5 m/sec

In [20]:
dataset_cities_results= dataset_cities[(dataset_cities.Rain <= 0.5) & (dataset_cities.Humidity <= 60) & (dataset_cities.Wind <= 5) & (dataset_cities.Temperature > 10)]
dataset_cities_results

,id,City,Latitude,Longitude,Temperature,Humidity,Wind,Rain
14,15,Annecy,45.8992348,6.1288847,15.000000,60.0,1.9,0.2
15,16,Grenoble,45.1875602,5.7357819,17.299999,57.0,2.1,0.4
17,18,Gorges du Verdon,43.7496562,6.3285616,11.900000,53.0,3.0,0.0
21,22,Aix en Provence,43.5298424,5.4474738,15.800000,48.0,3.7,0.0
23,24,Uzes,44.0121279,4.4196718,15.100000,57.0,4.3,0.0
24,25,Nimes,43.8374249,4.3600687,16.500000,53.0,4.3,0.0
27,28,Collioure,42.52505,3.0831554,17.400000,57.0,3.5,0.0
28,29,Carcassonne,43.2130358,2.3491069,16.700001,59.0,4.7,0.0


In [21]:
# sauvegarde liste des villes sélectionnées
dataset_cities_results.to_csv("dataset_cities_results.csv", index=False)

In [22]:
# conversion des données Latitude/Longitude en nb décimal pour pouvoir être utilisées dans la carte
dataset_cities_results["Latitude"]=dataset_cities_results.loc[:,"Latitude"].astype(float, errors="raise")
dataset_cities_results["Longitude"]=dataset_cities_results.loc[:,"Longitude"].astype(float, errors="raise")

C:\Users\SD\AppData\Local\Temp\ipykernel_28252\3358253806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cities_results["Latitude"]=dataset_cities_results.loc[:,"Latitude"].astype(float, errors="raise")
C:\Users\SD\AppData\Local\Temp\ipykernel_28252\3358253806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cities_results["Longitude"]=dataset_cities_results.loc[:,"Longitude"].astype(float, errors="raise")


In [23]:
# création du graphe
cities_results = px.scatter_mapbox(
    dataset_cities_results,
    title = "Localisation des villes à privilégier pour vos vacances",
    lon="Longitude",
    lat="Latitude",
    color="Temperature",
    mapbox_style="open-street-map",
    text="City",
    zoom=5,
)
cities_results.update_layout(width=1000,height = 900, margin={"r":0,"t":50,"l":0,"b":0}, title_x = 0.5, title_y = 0.98)
cities_results.update_traces(marker={'size': 15}) # permet de definir une taille fixe pour les points sur la carte
cities_results.show()

#### PART 3. Obtenir une liste d'hotels disponibles pour les destinations retenues à partir de booking.com

In [24]:
class booking_spider(scrapy.Spider):
    name = "booking"
    base_lien = []

    def start_requests(self):
        checkin_date = date.today() + timedelta(days=1)
        checkout_date = checkin_date + timedelta(days=1)    
        for city in dataset_cities_results["City"] :
            id = dataset_cities_results.loc[dataset_cities_results.City==city,"id"].values[0]
            parameters_booking = {"ss":city, "no_rooms":"1", "checkin":checkin_date, "checkout":checkout_date, "group_adults":"2", "group_children":"0"}
            final_url =f'{"https://www.booking.com/searchresults.fr.html?"}{urllib.parse.urlencode(parameters_booking)}'
            yield scrapy.Request(url=final_url, callback=self.parse, cb_kwargs={"id" : id})

    def parse(self, response, id):
        links = response.xpath('//h3[@class="aab71f8e4e"]/a/@href').getall()
        for lien in links :
            lien = str(lien).split("?")[0]
            self.base_lien.append(lien)
            yield scrapy.Request(url=lien, callback=self.parse_detail, cb_kwargs={"id" : id})

    def parse_detail(self, response, id) :
        yield {
            'id' : id,
            'Name' : response.xpath('//h2[@class="d2fee87262 pp-header__title"]/text()').get(),
            'rating' : response.xpath('//div[@class="a3b8729ab1 d86cee9b25"]/text()').get(),
            'gps' : response.xpath('//a[@id="hotel_address"]/@data-atlas-latlng').get(""),
            'url' : response.request.url,
            'Comment' : response.xpath('//p[@class="a53cbfa6de b3efd73f69"]/text()').get()
        }
            
filename = "booking.csv"

if filename in os.listdir("C:/Users/SD\Desktop/Cours/Jedha/Fullstack_Datascience/4 - Data_Collection_and_Management/5 - Projet/"):
        os.remove("C:/Users/SD\Desktop/Cours/Jedha/Fullstack_Datascience/4 - Data_Collection_and_Management/5 - Projet/" + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
    'LOG_LEVEL': logging.INFO,
    'FEED_EXPORT_ENCODING' : "utf-8", # résoud le pb de caractères
    "FEEDS": {filename : {"format": "csv"},
    }
})

process.crawl(booking_spider)
process.start()

2024-03-11 11:29:34 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-03-11 11:29:34 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Windows-10-10.0.22631-SP0
2024-03-11 11:29:34 [scrapy.addons] INFO: Enabled addons:
[]
2024-03-11 11:29:34 [py.warnings] WARNING: C:\Users\SD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning:

'2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compati

2024-03-11 11:29:35 [scrapy.extensions.telnet] INFO: Telnet Password: da634122c1043261
2024-03-11 11:29:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2024-03-11 11:29:35 [scrapy.crawler] INFO: Overridden settings:
{'FEED_EXPORT_ENCODING': 'utf-8',
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) '
               'Gecko/20100101 Firefox/114.0'}
2024-03-11 11:29:36 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirec

In [25]:
# chargement du dataset comprenant les hotels
dataset_hotels = pd.read_csv("booking.csv")

In [26]:
dataset_hotels["id"].value_counts()

id
25    25
29    25
28    25
16    25
15    25
22    25
18    25
24    24
Name: count, dtype: int64

In [27]:
# conversion de la colonne "rating" au bon format, en remplaçant les virgules par des points
dataset_hotels["rating"] = dataset_hotels["rating"].str.replace(",",".").astype(dtype="float", errors="raise")

In [28]:
# création de deux colonnes latitude et longitude afin de permettre la visualisaiton sur une carte
dataset_hotels["latitude"] = dataset_hotels["gps"].apply(lambda x : x.split(",")[0]).astype(dtype="float", errors="raise")
dataset_hotels["longitude"] = dataset_hotels["gps"].apply(lambda x : x.split(",")[1]).astype(dtype="float", errors="raise")

In [47]:
# overview of the dataset
dataset_hotels.head(10)

,id,Name,rating,url,Comment,latitude,longitude
0,25,Les Madones,8.5,https://www.booking.com/hotel/fr/les-madones-n...,"Situé à Nîmes, à moins de 29 km du parc des ex...",43.764623,4.360898
1,25,Abyad - A 300 m des Arènes,7.2,https://www.booking.com/hotel/fr/abyad-booking...,Idéalement situé dans le centre-ville de Nîmes...,43.836847,4.362033
2,25,Royal Hôtel,8.3,https://www.booking.com/hotel/fr/royal.fr.html,"Situé au cœur de Nîmes, le Royal Hôtel dispose...",43.838815,4.354818
3,29,**Auberge de l'Aude**,8.5,https://www.booking.com/hotel/fr/auberge-de-l-...,"Offrant une vue sur la cour intérieure, **Aube...",43.209820,2.348549
4,25,Hotel Mas De Galoffre,7.6,https://www.booking.com/hotel/fr/mas-de-galoff...,Situé à 5 minutes en voiture du centre de Nîme...,43.788452,4.353842
5,25,Zenitude Hôtel-Résidences Nîmes Centre,7.9,https://www.booking.com/hotel/fr/adagio-access...,Le Zenitude Hôtel-Résidences Nîmes Centre est ...,43.832493,4.367857
6,25,Empire Hôtel,7.4,https://www.booking.com/hotel/fr/empire-nimes....,"L'Empire Hôtel est situé au cœur de Nîmes, en ...",43.840301,4.363149
7,25,Studio Nîmes Centre,8.3,https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,43.839763,4.356942
8,25,Margaret - Hôtel Chouleur,9.3,https://www.booking.com/hotel/fr/margaret-chou...,"Idéalement situé à Nîmes, le Margaret - Hôtel ...",43.837117,4.357898
9,28,Appartement unique! 3 pieces au coeur de Colli...,6.6,https://www.booking.com/hotel/fr/appartement-u...,Appartement unique! Le 3 pieces au coeur de Co...,42.525772,3.080866


In [29]:
# création du graphe
hotels_results = px.scatter_mapbox(
    dataset_hotels,
    title = "Localisation des hôtels dans les villes à privilégier pour vos vacances",
    lon="longitude",
    lat="latitude",
    color="rating",
    mapbox_style="open-street-map",
    text="Name",
    zoom=5,
)
hotels_results.update_layout(width=1000,height = 900, margin={"r":0,"t":50,"l":0,"b":0}, title_x = 0.5, title_y = 0.98)
hotels_results.update_traces(marker={'size': 15}) # permet de definir une taille fixe pour les points sur la carte
hotels_results.show()

In [30]:
# sauvegarde liste des hotels nettoyée
dataset_hotels.to_csv("dataset_hotels.csv", index=False)

In [31]:
# fonction créée avec l'aide d'exemple trouvés sur internet (notamment stackoverfflow) mais je n'ai malheuresuement pas noté le lien

def read_credentials(file_path):
    """
    fonction qui permet d'aller chercher les codes d'accès à AWS dans un fichier txt situé au même emplacement que le script
    Args:
        file_path (string): chemin d'accès et nom du fichier
    Returns:
        un dictionnaire avec les credentials sous forme de clé(s)/valeur(s)
    """
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            credentials[key] = value
    return credentials

# Détermination du chemin et/ou nom du fichier où sont les credentials
credentials_path = "access.txt"

# Lecture des credentials à partir du fichier
credentials = read_credentials(credentials_path)

# Affectation à deux variables des credentials
access_key = credentials["access_key"]
secret_key = credentials["secret_key"]

In [32]:
# sauvegarde des fichiers sur S3
session = boto3.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key)

# créer une session "ressource"
s3 = session.resource("s3")

# connection a un bucket existant sur AWS-S3
bucket = s3.Bucket("myprojet-kayak") 

# mettre les fichiers dans le bucket créé sur S3
bucket.upload_file("dataset_cities.csv","dataset_cities.csv")
bucket.upload_file("booking.json","booking.json")
bucket.upload_file("dataset_hotels.csv","dataset_hotels.csv")

#### PART 4. RDS

In [ ]:
# !pip install psycopg2-binary

In [34]:
# Importation librairies
from sqlalchemy import create_engine, text, Column, Integer, String, Float, URL, select
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.sql import text
import psycopg2

#### A. Création des tables

In [70]:
# chemin et nom du fichier où se trouve les données d'accès
credentials_path = "rds.txt"

# utlisation de la fonction read_credentials
credentials = read_credentials(credentials_path)

In [71]:
# Creation d'une connection à RDS
url_rds = URL.create(
    "postgresql+psycopg2",
    username=credentials["username"],
    password=credentials["mdp"],
    host=credentials["hostname"],
    port = credentials["port"],
    database= "postgres"
)
engine = create_engine(url_rds, echo=True)

In [52]:
# instanciation d'une base declarative qui sera utlisée ensuite
Base = declarative_base()

In [53]:
# création d'une première table qui va accueillir les données concernant les villes 

class User(Base):
    __tablename__ = "data_citiesv2"
    #__table_args__ = {'extend_existing': True}

    # chaque paramètre correspond à une colonne dans la table, avec le format de la donnée
    id = Column(Integer, primary_key=True)
    City = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Temperature = Column(Float)
    Humidity = Column(Float)
    Wind = Column(Float)
    Rain = Column(Float)

    def __repr__(self):
        return "<User(City='{}', Latitude='{}', Longitude='{}', Temperature='{}', Humidity='{}', Wind='{}', Rain='{}')>".format(self.City, self.Latitude, self.Longitude, self.Temperature, self.Humidity, self.Wind, self.Rain)

In [54]:
# Creation de la 1ère table
Base.metadata.create_all(engine)

2024-03-11 12:33:07,191 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2024-03-11 12:33:07 [sqlalchemy.engine.Engine] INFO: select pg_catalog.version()


2024-03-11 12:33:07,193 INFO sqlalchemy.engine.Engine [raw sql] {}


2024-03-11 12:33:07 [sqlalchemy.engine.Engine] INFO: [raw sql] {}


2024-03-11 12:33:07,285 INFO sqlalchemy.engine.Engine select current_schema()


...

2024-03-11 12:33:07 [sqlalchemy.engine.Engine] INFO: 
CREATE TABLE data_citiesv2 (
	id SERIAL NOT NULL, 
	"City" VARCHAR, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT, 
	"Temperature" FLOAT, 
	"Humidity" FLOAT, 
	"Wind" FLOAT, 
	"Rain" FLOAT, 
	PRIMARY KEY (id)
)




2024-03-11 12:33:07,577 INFO sqlalchemy.engine.Engine [no key 0.00241s] {}


2024-03-11 12:33:07 [sqlalchemy.engine.Engine] INFO: [no key 0.00241s] {}


2024-03-11 12:33:07,658 INFO sqlalchemy.engine.Engine COMMIT


2024-03-11 12:33:07 [sqlalchemy.engine.Engine] INFO: COMMIT


In [55]:
# Insertion de notre dataframe sur les villes dans la table créée auparavant, en y ajoutant les données
dataset_cities.to_sql("data_citiesv2", engine, if_exists="append", index=False)

2024-03-11 12:33:29,597 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-03-11 12:33:29 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2024-03-11 12:33:29,602 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2024-03-11 12:33:29 [sqlalchemy.engine.Engine] INFO: SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2024-03-11 12:33:29,604 INFO sqlalchemy.engine.Engine [cached since 22.13s ago] {'table_name': 'data_citiesv2', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}


...

2024-03-11 12:33:29,716 INFO sqlalchemy.engine.Engine [generated in 0.00024s (insertmanyvalues) 1/1 (unordered)] {'id__0': 1, 'City__0': 'Mont Saint Michel', 'Temperature__0': 13.800000190734863, 'Longitude__0': '-1.511459954959514', 'Humidity__0': 82.0, 'Latitude__0': '48.6359541', 'Wind__0': 6.400000095367432, 'Rain__0': 0.699999988079071, 'id__1': 2, 'City__1': 'St Malo', 'Temperature__1': 13.600000381469727, 'Longitude__1': '-2.0260409', 'Humidity__1': 79.0, 'Latitude__1': '48.649518', 'Wind__1': 7.099999904632568, 'Rain__1': 0.20000000298023224, 'id__2': 3, 'City__2': 'Bayeux', 'Temperature__2': 14.100000381469727, 'Longitude__2': '-0.7024738', 'Humidity__2': 78.0, 'Latitude__2': '49.2764624', 'Wind__2': 6.599999904632568, 'Rain__2': 0.800000011920929, 'id__3': 4, 'City__3': 'Le Havre', 'Temperature__3': 11.399999618530273, 'Longitude__3': '0.1079732', 'Humidity__3': 87.0, 'Latitude__3': '49.4938975', 'Wind__3': 6.400000095367432, 'Rain__3': 1.899999976158142, 'id__4': 5, 'City__4

2024-03-11 12:33:29 [sqlalchemy.engine.Engine] INFO: [generated in 0.00024s (insertmanyvalues) 1/1 (unordered)] {'id__0': 1, 'City__0': 'Mont Saint Michel', 'Temperature__0': 13.800000190734863, 'Longitude__0': '-1.511459954959514', 'Humidity__0': 82.0, 'Latitude__0': '48.6359541', 'Wind__0': 6.400000095367432, 'Rain__0': 0.699999988079071, 'id__1': 2, 'City__1': 'St Malo', 'Temperature__1': 13.600000381469727, 'Longitude__1': '-2.0260409', 'Humidity__1': 79.0, 'Latitude__1': '48.649518', 'Wind__1': 7.099999904632568, 'Rain__1': 0.20000000298023224, 'id__2': 3, 'City__2': 'Bayeux', 'Temperature__2': 14.100000381469727, 'Longitude__2': '-0.7024738', 'Humidity__2': 78.0, 'Latitude__2': '49.2764624', 'Wind__2': 6.599999904632568, 'Rain__2': 0.800000011920929, 'id__3': 4, 'City__3': 'Le Havre', 'Temperature__3': 11.399999618530273, 'Longitude__3': '0.1079732', 'Humidity__3': 87.0, 'Latitude__3': '49.4938975', 'Wind__3': 6.400000095367432, 'Rain__3': 1.899999976158142, 'id__4': 5, 'City__4'

2024-03-11 12:33:29,775 INFO sqlalchemy.engine.Engine COMMIT


2024-03-11 12:33:29 [sqlalchemy.engine.Engine] INFO: COMMIT


35

In [ ]:
# suppression de la colonne "gps" inutile
dataset_hotels = dataset_hotels.drop("gps", axis=1)

In [60]:
# création d'une deuxième table qui va accueillir les données concernant les hôtels

class User(Base):
    __tablename__ = "data_hotelsv2"
    #__table_args__ = {'extend_existing': True}

    # chaque paramètre correspond à une colonne dans la table, avec le format de la donnée
    id = Column(Integer, primary_key=True) # permettra de faire la jointure avec la table cities
    Name = Column(String)
    Rating = Column(Float)
    Url = Column(String)
    Comment = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)

    def __repr__(self):
        return "<User(Name='{}', Rating='{}', Url='{}', Comment='{}', Latitude='{}', Longitude='{}')>".format(self.Name, self.Rating, self.Url, self.Comment, self.Latitude, self.Longitude)

2024-03-11 12:35:03 [py.warnings] WARNING: C:\Users\SD\AppData\Local\Temp\ipykernel_28252\882514365.py:3: SAWarning:

This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.




In [61]:
# Insertion de notre dataframe sur les hôtels dans la table créée auparavant, en y ajoutant les données
dataset_hotels.to_sql("data_hotelsv2", engine, if_exists="append", index=False)

2024-03-11 12:35:05,829 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-03-11 12:35:05 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2024-03-11 12:35:05,840 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2024-03-11 12:35:05 [sqlalchemy.engine.Engine] INFO: SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s


2024-03-11 12:35:05,848 INFO sqlalchemy.engine.Engine [cached since 118.4s ago] {'table_name': 'data_hotelsv2', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}


...

2024-03-11 12:35:06,037 INFO sqlalchemy.engine.Engine [generated in 0.00079s (insertmanyvalues) 1/1 (unordered)] {'rating__0': 8.5, 'longitude__0': 4.360898, 'id__0': 25, 'url__0': 'https://www.booking.com/hotel/fr/les-madones-nimes1.fr.html', 'Name__0': 'Les Madones', 'latitude__0': 43.7646229, 'Comment__0': "Situé à Nîmes, à moins de 29 km du parc des expositions et à 37 km de l'amphithéâtre d'Arles, l'établissement Les Madones propose des hébergements av ... (259 characters truncated) ... une télévision à écran plat, d'une cuisine entièrement équipée et d'une terrasse.\n\nL'aéroport de Nîmes-Garons, le plus proche, est implanté à 5 km.", 'rating__1': 7.2, 'longitude__1': 4.362033, 'id__1': 25, 'url__1': 'https://www.booking.com/hotel/fr/abyad-booking-cosy-dans-l-hyper-centre-de-nimes.fr.html', 'Name__1': 'Abyad - A 300 m des Arènes', 'latitude__1': 43.8368472, 'Comment__1': "Idéalement situé dans le centre-ville de Nîmes, l'Abyad - A 300 mètres des Arènes se trouve à 34 km de l'amph

2024-03-11 12:35:06 [sqlalchemy.engine.Engine] INFO: [generated in 0.00079s (insertmanyvalues) 1/1 (unordered)] {'rating__0': 8.5, 'longitude__0': 4.360898, 'id__0': 25, 'url__0': 'https://www.booking.com/hotel/fr/les-madones-nimes1.fr.html', 'Name__0': 'Les Madones', 'latitude__0': 43.7646229, 'Comment__0': "Situé à Nîmes, à moins de 29 km du parc des expositions et à 37 km de l'amphithéâtre d'Arles, l'établissement Les Madones propose des hébergements av ... (259 characters truncated) ... une télévision à écran plat, d'une cuisine entièrement équipée et d'une terrasse.\n\nL'aéroport de Nîmes-Garons, le plus proche, est implanté à 5 km.", 'rating__1': 7.2, 'longitude__1': 4.362033, 'id__1': 25, 'url__1': 'https://www.booking.com/hotel/fr/abyad-booking-cosy-dans-l-hyper-centre-de-nimes.fr.html', 'Name__1': 'Abyad - A 300 m des Arènes', 'latitude__1': 43.8368472, 'Comment__1': "Idéalement situé dans le centre-ville de Nîmes, l'Abyad - A 300 mètres des Arènes se trouve à 34 km de l'amphi

2024-03-11 12:35:06,235 INFO sqlalchemy.engine.Engine COMMIT


2024-03-11 12:35:06 [sqlalchemy.engine.Engine] INFO: COMMIT


199

#### B. Test de requêtes sur les tables

In [111]:
# création d'une connexion à la base de données en utilisant l'objet engine
conn = engine.connect()

In [106]:
# 1ère requête sur la table Cities
statement = text("SELECT * \
                FROM data_citiesv2 \
                WHERE id>10 AND id<20")
result = conn.execute(statement)
result.fetchall()

2024-03-11 13:16:56,103 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-03-11 13:16:56 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2024-03-11 13:16:56,106 INFO sqlalchemy.engine.Engine SELECT *                 FROM data_citiesv2                 WHERE id>10 AND id<20


2024-03-11 13:16:56 [sqlalchemy.engine.Engine] INFO: SELECT *                 FROM data_citiesv2                 WHERE id>10 AND id<20


2024-03-11 13:16:56,108 INFO sqlalchemy.engine.Engine [cached since 1278s ago] {}


2024-03-11 13:16:56 [sqlalchemy.engine.Engine] INFO: [cached since 1278s ago] {}


[(11, 'Colmar', 48.0777517, 7.3579641, 14.600000381469727, 61.0, 2.5999999046325684, 0.5),
 (12, 'Eguisheim', 48.0447968, 7.3079618, 14.300000190734863, 61.0, 2.5, 0.30000001192092896),
 (13, 'Besancon', 47.2380222, 6.0243622, 11.899999618530273, 82.0, 3.4000000953674316, 1.2000000476837158),
 (14, 'Dijon', 47.3215806, 5.0414701, 11.300000190734863, 88.0, 3.5, 1.5),
 (15, 'Annecy', 45.8992348, 6.1288847, 15.0, 60.0, 1.899999976158142, 0.20000000298023224),
 (16, 'Grenoble', 45.1875602, 5.7357819, 17.299999237060547, 57.0, 2.0999999046325684, 0.4000000059604645),
 (17, 'Lyon', 45.7578137, 4.8320114, 12.699999809265137, 65.0, 2.299999952316284, 0.10000000149011612),
 (18, 'Gorges du Verdon', 43.7496562, 6.3285616, 11.899999618530273, 53.0, 3.0, 0.0),
 (19, 'Bormes les Mimosas', 43.1506968, 6.3419285, 14.399999618530273, 63.0, 4.099999904632568, 0.0)]

In [112]:
# 2ème requête sur la table hôtels
statement = text("SELECT * \
                FROM data_hotelsv2 \
                WHERE id=25 \
                ORDER BY Rating DESC \
                LIMIT 10")
result = conn.execute(statement)
result.fetchall()

2024-03-11 13:19:42,426 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-03-11 13:19:42 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2024-03-11 13:19:42,429 INFO sqlalchemy.engine.Engine SELECT *                 FROM data_hotelsv2                 WHERE id=25                 ORDER BY Rating DESC                 LIMIT 10


2024-03-11 13:19:42 [sqlalchemy.engine.Engine] INFO: SELECT *                 FROM data_hotelsv2                 WHERE id=25                 ORDER BY Rating DESC                 LIMIT 10


2024-03-11 13:19:42,434 INFO sqlalchemy.engine.Engine [cached since 10.57s ago] {}


2024-03-11 13:19:42 [sqlalchemy.engine.Engine] INFO: [cached since 10.57s ago] {}


[(25, 'Margaret - Hôtel Chouleur', 9.3, 'https://www.booking.com/hotel/fr/margaret-chouleur.fr.html', "Idéalement situé à Nîmes, le Margaret - Hôtel Chouleur propose des chambres climatisées, une piscine extérieure ouverte en saison, une connexion Wi-F ... (716 characters truncated) ... éroport le plus proche, celui de Nîmes-Garons, est situé à 9 km. Un service de navette aéroport peut être assuré moyennant des frais supplémentaires.", 43.837117, 4.357898),
 (25, 'Maison Albar Hotels L’Imperator', 9.1, 'https://www.booking.com/hotel/fr/imperator-nimes.fr.html', "La Maison Albar Hotels L’Imperator est un hôtel de luxe situé dans la ville de Nîmes, en bordure du jardin de la Fontaine et à moins de 5 minutes à p ... (1438 characters truncated) ... \n\nUn parking privé est accessible sur place, sur réservation. La gare TGV de Nîmes se trouve à moins de 2 km de la Maison Albar Hotels L’Imperator.", 43.83866613, 4.35276791),
 (25, 'Odalys City Nîmes Arènes', 8.8, 'https://www.booking.com/hot

#### PART 5. Pour aller plus loin

Avec plus de temps, nous aurions souhaiter notamment :
- mieux documenter les fonctions (descriptif des variables, objectif de la fonction)
- nettoyer les données scrapées du site booking.com, et les présenter sous un format plus adaptées
- filtrer le nombre d'hôtels renvoyés afin d'en limiter le nombre, à partir d'un ou plusieurs critères (par ex top 5 rating ou prix)
- approfondir le travail concernant la création de base de données avec RDS
- créer un dashboard avec streamlit par exemple afin de présenter l'ensemble du travail